In [1]:
!pip install m2cgen
!pip install skl2onnx
!pip install sklearn-porter

     |████████████████████████████████| 73 kB 614 kB/s 
You should consider upgrading via the '/opt/conda/bin/python3.7 -m pip install --upgrade pip' command.
     |████████████████████████████████| 191 kB 1.3 MB/s 
     |████████████████████████████████| 64 kB 1.8 MB/s 
You should consider upgrading via the '/opt/conda/bin/python3.7 -m pip install --upgrade pip' command.
     |████████████████████████████████| 144 kB 1.3 MB/s 
You should consider upgrading via the '/opt/conda/bin/python3.7 -m pip install --upgrade pip' command.


In [2]:
import cv2
import os
from tqdm import *
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import math
import random
import urllib
import requests
import webbrowser
import shutil
import collections
import re
import time
import json
from glob import glob
from PIL import Image
import pickle

import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import *
from tensorflow.keras.layers import *
import tensorflow.keras.layers as layers
from tensorflow.keras.initializers import *
from tensorflow.keras.models import *
from tensorflow.keras.optimizers import *
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import *

from sklearn.utils import *
from sklearn.preprocessing import *
from sklearn.model_selection import *
from sklearn.metrics import *
from sklearn import svm
from sklearn.neighbors import KNeighborsClassifier

import keras.backend as K
from scipy.stats import *
import lightgbm as lgb
#import optuna.integration.lightgbm as lgb
import xgboost as xgb

import m2cgen as m2c
from sklearn_porter import Porter

print('Using Tensorflow version:', tf.__version__)

Using Tensorflow version: 2.3.1


/opt/conda/lib/python3.7/site-packages/sklearn/utils/deprecation.py:143: FutureWarning: The sklearn.tree.tree module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.tree. Anything that cannot be imported from sklearn.tree is now part of the private API.
  warnings.warn(message, FutureWarning)
/opt/conda/lib/python3.7/site-packages/sklearn/utils/deprecation.py:143: FutureWarning: The sklearn.ensemble.weight_boosting module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.ensemble. Anything that cannot be imported from sklearn.ensemble is now part of the private API.
  warnings.warn(message, FutureWarning)
/opt/conda/lib/python3.7/site-packages/sklearn/utils/deprecation.py:143: FutureWarning: The sklearn.ensemble.forest module is  deprecated in version 0.22 and will be removed in version 0.24. Th

In [3]:
imgSize = 28
BS = 1024
EPOCHS = 10
INIT_LR = 3e-4
opt = RMSprop(lr=0.001, rho=0.9, epsilon=1e-08, decay=0.0)

In [4]:
trainAug = ImageDataGenerator(rotation_range=5, rescale=1./255, validation_split=0.2)
validAug = ImageDataGenerator(rescale=1./255, validation_split=0.3)

In [5]:
train_generator = trainAug.flow_from_directory('../input/mnistasjpg/trainingSet/trainingSet/',
                                               target_size=(imgSize, imgSize),
                                               color_mode='grayscale',
                                               class_mode='categorical',
                                               batch_size=BS, 
                                               shuffle=False,
                                               subset='training')
valid_generator = validAug.flow_from_directory('../input/mnistasjpg/trainingSet/trainingSet/',
                                               target_size=(imgSize, imgSize),
                                               color_mode='grayscale',
                                               class_mode='categorical',
                                               batch_size=BS, 
                                               shuffle=False,
                                               subset='validation')

Found 33604 images belonging to 10 classes.
Found 12596 images belonging to 10 classes.


In [6]:
model = Sequential()
model.add(Flatten())

In [7]:
X_train = model.predict(train_generator, verbose = 1)
Y_train = train_generator.classes
X_valid = model.predict(valid_generator, verbose = 1)
Y_valid = valid_generator.classes

13/13 [==============================] - 42s 3s/step


In [8]:
from sklearn.linear_model import LogisticRegression
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC

model = SVC()

In [9]:
model.fit(X_train, Y_train)

SVC()

In [10]:
np.mean(model.predict(X_valid)==Y_valid)

0.9810257224515719

In [11]:
from skl2onnx import convert_sklearn
from skl2onnx.common.data_types import FloatTensorType
initial_type = [('float_input', FloatTensorType([None, 4]))]
onx = convert_sklearn(model, initial_types=initial_type)
with open("rf_iris.onnx", "wb") as f:
    f.write(onx.SerializeToString())